In [ ]:
import glob
from collections import OrderedDict

import ROOT
ROOT.gROOT.SetBatch(True)
from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F, TH1D, TH2D, THStack, TF1
from ROOT import gBenchmark, gRandom, gSystem, Double, gPad, TFitResultPtr, TMath
import root_numpy
import psutil
import pandas
import math
#matplotlib.use('agg')
%matplotlib inline
import matplotlib #as matplot
print(matplotlib.__version__)
#print(matplotlib.path)
import matplotlib.pyplot as plt
from matplotlib import cm as cm
import numpy as np
import time
from datetime import datetime

from sklearn.metrics import roc_curve, auc
import pylab
import sklearn as sk
print(sk.__version__)
from sklearn.model_selection import train_test_split

import tensorflow as tf
print(tf.__version__)
######################
import keras as kr
from keras.models import Sequential
from keras.layers import InputLayer, Input
from keras.layers import Reshape, MaxPooling2D
from keras.layers import Conv2D, Dense, Flatten, Dropout, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU, ELU, PReLU 
from keras.optimizers import Adamax
from keras.optimizers import Nadam
print(kr.__version__)
#from keras import backend as K
import tensorflow as tf
print(tf.__version__)
config = tf.ConfigProto(intra_op_parallelism_threads=32, \
                        inter_op_parallelism_threads=32, \
                        allow_soft_placement=True, \
                        device_count = {'CPU': 32}
                       )
session = tf.Session(config=config)
#K.set_session(session)
## it will issue a warning, just ignore

In [ ]:
# test that matplotlib imported ok
x1, y1 = [-1, 12], [1, 4]
x2, y2 = [1, 10], [3, 2]
plt.plot(x1, y1, x2, y2, marker = 'o')
plt.show()

In [ ]:
channel='2l_2tau_HH'

startTime = datetime.now()
execfile("../python/data_manager.py") 

if channel=='2l_2tau_HH' : 
    execfile("../cards/info_2l_2tau_HH.py") 
    execfile("../cards/auxiliary_functions_2l_2tau_HH.py") ## Loading the functions and setup variables specific to 2l_2tau channels 
    execfile("../cards/NN_settings_2l_2tau_HH.py") ## Loading the boolean settings specific to 2l_2tau channel
    
log_file_name=channel+".log"
if 'evtLevelSUM_HH_2l_2tau_res' in bdtType:
    file1_ = open(log_file_name, 'w+')
else: 
    file1_ = open('roc.log','w+')    


import shutil,subprocess
proc=subprocess.Popen(['mkdir '+channel],shell=True,stdout=subprocess.PIPE)
out = proc.stdout.read()    
    
    
output = read_from(Bkg_mass_rand, tauID)

In [ ]:
print ("read from:", output["inputPath"])
print ("Date: ", time.asctime( time.localtime(time.time()) ))

if(TrainMode == 0): ## All Masses included in training 
    data=load_data_2017(
            output["inputPath"],
            output["channelInTree"],
            trainVars(True),
            [],
            bdtType,
            channel,
            output["keys"],
            output["masses"],
            output["mass_randomization"],
            )
    mass_list = output["masses"]
    test_masses = output["masses_test"]
elif(TrainMode == 1): ## Only Low Masses (<= 400 GeV) included in training                                                                                                                                                                                              
        data=load_data_2017(
            output["inputPath"],
            output["channelInTree"],
            trainVars(True),
            [],
            bdtType,
            channel,
            output["keys"],
            output["masses_low"],
            output["mass_randomization"],
            )
        mass_list = output["masses_low"]
        test_masses = output["masses_test_low"]
elif(TrainMode == 2): ## Only High Masses (> 400 GeV) included in training                                                                                                                                                                                              
        data=load_data_2017(
            output["inputPath"],
            output["channelInTree"],
            trainVars(True),
            [],
            bdtType,
            channel,
            output["keys"],
            output["masses_high"],
            output["mass_randomization"],
            )
        mass_list = output["masses_high"]
        test_masses = output["masses_test_high"]
else:        
    data=load_data_2017(
        output["inputPath"], 
        output["channelInTree"], 
        trainVars(True), 
        [], 
        bdtType, 
        channel,
        output["keys"], 
        output["masses"],
        output["mass_randomization"]
        ) # note: I had to add channel as argument of the function
    mass_list = output["masses"]
    test_masses = output["masses_test"]
    
    
#data.dropna(subset=["totalWeight"],inplace = True) ## Was used in the BDT code
#data.fillna(0)                                     ## Was used in the BDT code

print (len(data))
print ("Date: ", time.asctime( time.localtime(time.time()) ))

In [ ]:
plotAll=False
BDTvariables=trainVars(plotAll, variables, bdtType)

import copy

## Removing gen_mHH from the list of input variables                                                                                                                                                                                                                    
BDTvariables_wo_gen_mHH = copy.deepcopy(BDTvariables)  ## Works                                                                                                                                                                                                         
BDTvariables_wo_gen_mHH.remove("gen_mHH")

trainvar = variables

if channel=='2l_2tau_HH' :
    labelBKG = "TT+DY+VV"
else:
    labelBKG = "TT+DY+VV+W"
    
print("mass_list", mass_list)
print("test_masses", test_masses)

if((channel=='2l_2tau_HH') and do_ReweightVars):
        DoFits = True
        print DoFits
        print("Perfoming Fits to TProfile plots for signal")
        ## --- Making TProfile plots with fits (Signal) --- ###
        MakeTProfile_New(channel, data, BDTvariables_wo_gen_mHH, 1, DoFits, "before", TrainMode, mass_list)
else:
        DoFits = False
        print DoFits
        print("Not Perfoming Fits to TProfile plots for signal")

        
        

if((channel=='2l_2tau_HH') and do_2l_2tau_diagnostics == True):  
    ## --- Making TProfile plots w/o fits (background) --- ###                                                                                                                                           
    MakeTProfile_New(channel, data, BDTvariables_wo_gen_mHH, 0, False, "before", TrainMode, mass_list)        

    ## --- Making 1D Histo plots (background) --- ###                                                                                                                                                                                                                   
    MakeHisto1D_New(channel, data, BDTvariables, "before")

    ## --- Making 1D THStack plots (background) --- ###                                                                                                                                                                                                                 
    MakeTHStack_New(channel, data, BDTvariables, "before")
else:
    print("No plots will be made for 2l_2tau diagnostics")  

if((channel=='2l_2tau_HH') and do_ReweightVars):
    ## ----- SCALING I/P VAR.S IN DATA USING THE FITS DONE ABOVE---- ###                                                                                                                                                                                            
    ReweightDataframe_New(data, channel, BDTvariables_wo_gen_mHH, mass_list)
else:
    print("No plots and fits will be made for 2l_2tau diagnostics")
    
    
weights="evtWeight"

print ("Date: ", time.asctime( time.localtime(time.time()) ))
data["weight_cx"] = data[weights]
data["weight_train"] = data[weights]
data["NN_output"] = 1.0

print ("separate datasets odd/even")
data_even = data.loc[(data["event"].values % 2 == 0) ]
data_odd = data.loc[~(data["event"].values % 2 == 0) ]

order_train = [data_odd, data_even]
order_train_name = ["odd","even"]


print ("balance datasets by even/odd chunck")
for data_do in order_train :
    #### Normalization by cross section                                                                                                                                                                                                                                   
    for wei in ["weight_cx", "weight_train"] :
        if 'SUM_HH' in bdtType :
            data_do.loc[(data_do['key'].isin(['TTTo2L2Nu','TTToSemiLeptonic'])), [wei]]              *= output["TTdatacard"]/data_do.loc[(data_do['key'].isin(['TTTo2L2Nu','TTToSemiLeptonic'])), weights].sum()
            data_do.loc[(data_do['key']=='DY'), [wei]]                            *= output["DYdatacard"]/data_do.loc[(data_do['key']=='DY'), weights].sum()
            if "evtLevelSUM_HH_bb1l_res" in bdtType :
                data_do.loc[(data_do['key']=='W'), [wei]]                         *= Wdatacard/data_do.loc[(data_do['key']=='W')].sum() ## Saswati check please !!!                                                                                                       
        if "evtLevelSUM_HH_2l_2tau_res" in bdtType :
               #data_do.loc[(data_do['key']=='TTZJets'), [wei]]                       *= output["TTZdatacard"]/data_do.loc[(data_do['key']=='TTZJets'), weights].sum() ## TTZJets                                                                                          
               #data_do.loc[(data_do['key']=='TTWJets'), [wei]]                       *= output["TTWdatacard"]/data_do.loc[(data_do['key']=='TTWJets'), weights].sum() ## TTWJets + TTWW                                                                                   
               data_do.loc[(data_do['key']=='ZZ'), [wei]]                            *= output["ZZdatacard"]/data_do.loc[(data_do['key']=='ZZ'), weights].sum() ## ZZ +ZZZ                                                                                                
               data_do.loc[(data_do['key']=='WZ'), [wei]]                            *= output["WZdatacard"]/data_do.loc[(data_do['key']=='WZ'), weights].sum() ## WZ + WZZ_4F                                                                                            
               data_do.loc[(data_do['key']=='WW'), [wei]]                            *= output["WWdatacard"]/data_do.loc[(data_do['key']=='WW'), weights].sum() ## WW + WWZ + WWW_4F                                                                                      
               #data_do.loc[(data_do['key']=='VH'), [wei]]                        *= output["VHdatacard"]/data_do.loc[(data_do['key']=='VH'), weights].sum() # consider removing                                                                                          
               #data_do.loc[(data_do['key']=='TTH'), [wei]]                       *= output["TTHdatacard"]/data_do.loc[(data_do['key']=='TTH'), weights].sum() # consider removing    
    ### Normalize sig/BKG and do table of nevents/mass
    for mass in output["masses"] :
        data_do.loc[(data_do['target']==1) & (data_do["gen_mHH"] == mass),"weight_train"] *= 10000./data_do.loc[(data_do['target']==1) & (data_do["gen_mHH"]== mass), "weight_train"].sum() ## Changed from 1000
        data_do.loc[(data_do['target']==0) & (data_do["gen_mHH"] == mass),"weight_train"] *= 10000./data_do.loc[(data_do['target']==0) & (data_do["gen_mHH"]== mass), "weight_train"].sum() ## Changed from 1000 
    print ("Date: ", time.asctime( time.localtime(time.time()) ))

    print ("training statistics by mass")
    for mass in output["masses"] :
        print (
               str(mass)+": sig = "+\
               str(len(data_do.loc[(data['target']==1) & (data_do["gen_mHH"] == mass),["weight_train"]]))+\
               " BKG = "+str(len(data_do.loc[(data['target']==0) & (data_do["gen_mHH"] == mass),["weight_train"]]))
              )

    print ("\n norm by mass - test")
    for mass in output["masses"] :
        print (
               str(mass)+": sig = "+\
               str(data_do.loc[(data_do['target']==1) & (data_do["gen_mHH"] == mass),"weight_train"].sum())+\
               " BKG = "+str(data_do.loc[(data_do['target']==0) & (data_do["gen_mHH"] == mass),"weight_train"].sum())
              )

if((channel=='2l_2tau_HH') and do_2l_2tau_diagnostics == True):        
    ## ----Merging the odd and even data-sets ---------------------------------------####                                                                                                                                                                               
    ## ---(reweighting the merged dataframe by 0.5 as it is derived from 2 halves) ---###                                                                                                                                                                               
    data_odd_copy = data_odd.copy(deep=True) ## Making sure we do not alter the halves used for roc curves later                                                                                                                                                        
    data_even_copy = data_even.copy(deep=True) ## Making sure we do not alter the halves used for roc curves later                                                                                                                                                      
    data_do = data_odd_copy.append(data_even_copy, ignore_index=True)
    data_do.loc[data_do['target']==0, [weights]] *= 0.5
    data_do.loc[data_do['target']==1, [weights]] *= 0.5
    label = "after"

    ## --- Making TProfile plots w/o fitting (Signal) --- ###                                                                                                                                                                                                           
    MakeTProfile_New(channel, data, BDTvariables_wo_gen_mHH, 1, False, label, TrainMode, mass_list)

    ## --- Making TProfile plots (background) --- ###                                                                                                                                                                                                                   
    MakeTProfile_New(channel, data, BDTvariables_wo_gen_mHH, 0, False, label, TrainMode, mass_list)

    ## --- Making 1D Histo plots (background) --- ###                                                                                                                                                                                                                   
    MakeHisto1D_New(channel, data,  BDTvariables, label)

    ## --- Making 1D THStack plots (background) --- ###                                                                                                                                                                                                                 
    MakeTHStack_New(channel, data, BDTvariables, label)
else:
    print("No plots will be made for 2l_2tau diagnostics")    
    


In [ ]:
"""
Check of the resulting weights - the sizes of the training weight
"""
fig, ax = plt.subplots(figsize=(4, 4))
if(channel=='2l_2tau_HH'):
    keysToBKG = ['WW', 'WZ', 'ZZ', 'DY', 'TTTo2L2Nu', 'TTToSemiLeptonic'] # 'VH', 'TTH', 'TTToHadronic', 'TTZJets', 'TTWJets' 
else:
    keysToBKG = ['WW', 'WZ', 'ZZ', 'DY', 'TTTo2L2Nu', 'TTToSemiLeptonic', 'W']
    
#colors = ['cyan','orange','k','r','green','magenta','b',]
vars = ["weight_train"]#"multitarget"]

for kk, key in enumerate(keysToBKG) :
  for vv, var in enumerate(vars) : 
    ax.hist(
        np.array(data.loc[(data['key']==key), var].values,dtype='float64'), # 
        weights=data.loc[(data['key']==key), "evtWeight"], # "weight_train_cat"
        range=(-1.0,10.),bins=40, histtype='step', normed=True, lw=2, 
        label=key
    )
    ax.set_xlabel(var)
ax.legend(loc="best", title= channel)

In [ ]:
## load the variables
#trainvar = trainVars(False, "testVars2")
#print trainvar
trainvar = trainVars(False, variables, bdtType) ## = BDTvariables
print trainvar

In [ ]:
"""
Draw some plots on lists of variables for BKG
"""
#keysToBKG = ['TTZJets', 'WZ', 'ZZ', 'DY', 'VH', 'ttH', 'TTTo2L2Nu']
 
## i try to do 3 X 3 plots (= enter up to nine entries in each sublist)
## try to add strictly decreasing variables as first in each sublist, better for the legend positioning
#listdraw = [
#    [ 'tau1_pt', 'tau2_pt', 'mT_lep1', 'mT_lep2', 'met', 'm_ll', 'mTauTau', 'diHiggsVisMass', 'diHiggsMass' ], 
#    ['nBJet_medium', 'nElectron', 'max_tau_eta', 'max_lep_eta', 'gen_mHH'],
#]

if(channel=='2l_2tau_HH'):
    listdraw = [
        ['diHiggsMass', 'diHiggsVisMass', 'tau1_pt', 'nBJet_medium', 'nElectron', 'dr_lep_tau_min_SS', 'met_LD', 'tau2_pt', 'dr_lep_tau_min_OS'],
        ['gen_mHH'],
               ]
else:
    listdraw = [
        ['diHiggsMass', 'diHiggsVisMass', 'tau1_pt', 'nBJet_medium', 'nElectron', 'dr_lep_tau_min_SS', 'met_LD', 'tau2_pt', 'dr_lep_tau_min_OS'],
        ['gen_mHH'],
               ]

for featuresDraw in listdraw:
    sizeArray=int(math.sqrt(len(featuresDraw))) if math.sqrt(len(featuresDraw)) % int(math.sqrt(len(featuresDraw))) == 0 else int(math.sqrt(len(featuresDraw)))+1
    plt.figure(figsize=(4*sizeArray,4*sizeArray))
    for n, feature in enumerate(featuresDraw) :
        min_value, max_value = np.percentile(data[feature], [0.0, 99])
        # fig, ax = plt.subplots(figsize=(4, 4))
        plt.subplot(sizeArray, sizeArray, n+1)
        for kk, key in enumerate(keysToBKG) :
            if 'TTZJets' in key or 'TTWJets' in key : linestyle = "--"
            else :linestyle = "-"
            plt.hist(
            np.array(data.loc[(data['key']==key), feature].values,dtype='float64'), 
            weights=data.loc[(data['key']==key) , "evtWeight"], 
            range=(min_value, max_value), 
            bins=12, histtype='step', ls=linestyle, 
            normed=True, lw=2, #color=colors[kk],
            label=key
            )
            plt.xlabel(feature)
        if n == 0 : plt.legend(loc="upper right", title= channel)

In [ ]:
for featuresDraw in listdraw:
    sizeArray=int(math.sqrt(len(featuresDraw))) if math.sqrt(len(featuresDraw)) % int(math.sqrt(len(featuresDraw))) == 0 else int(math.sqrt(len(featuresDraw)))+1
    plt.figure(figsize=(4*sizeArray,4*sizeArray))
    for n, feature in enumerate(featuresDraw) :
        min_value, max_value = np.percentile(data[feature], [0.0, 99])
        # fig, ax = plt.subplots(figsize=(4, 4))
        plt.subplot(sizeArray, sizeArray, n+1)
        for mass in [300,400,700] :
            plt.hist(
            np.array(data.loc[(data["gen_mHH"] == mass), feature].values,dtype='float64'), 
            weights=data.loc[(data["gen_mHH"] == mass) , "evtWeight"], 
            range=(min_value, max_value), 
            bins=10, histtype='step', ls=linestyle, 
            normed=True, lw=2, #color=colors[kk],
            label="mass = "+str(mass)
            )
        plt.hist(
        np.array(data.loc[(data['target']==0), feature].values,dtype='float64'), 
        weights=data.loc[(data['target']==0) , "evtWeight"], 
        range=(min_value, max_value), 
        bins=10, histtype='step', ls='--', 
        normed=True, lw=3, color='k',
        label="BKG"
        )
        plt.xlabel(feature)
        if n == 0 : plt.legend(loc="upper right", title= channel)

In [ ]:
# create model -- for binary activation='sigmoid'
nclasses = 1

## ---- Set these values for NN hyper-parameters (for gen_mHH as Input Var.) --- ##                                                                                                                                                                                                                     
EPOCHS=100 # 50 is more optimum
DROPOUT=0.05 # 0.01 is more optimum
LR=0.00005
SCHEDULE_DECAY=0.000005
BATCH_SIZE=256 
#EPOCHS=35
#DROPOUT=0.01
#LR=0.0005
#SCHEDULE_DECAY=0.00005
#BATCH_SIZE=256 ## 32,256
##----------------------------------------------------------##       

## ---- Set these values for NN hyper-parameters (w/o gen_mHH as Input Var.) --- ##                                                                                                                                                                                                                     
EPOCHS2=35
DROPOUT2=0.01
LR2=0.0005
SCHEDULE_DECAY2=0.00005
BATCH_SIZE2=256 ## 32,256
##----------------------------------------------------------##       


hyppar=str(variables)+"_epochs_"+str(EPOCHS)+"_dropout_"+num_to_str(DROPOUT)+"_lr_"+num_to_str(LR)+"_sch_decay_"+num_to_str(SCHEDULE_DECAY)+"_batch_size_"+str(BATCH_SIZE)
hyppar2=str(variables)+"_wo_gen_mHH"+"_epochs_"+str(EPOCHS2)+"_dropout_"+num_to_str(DROPOUT2)+"_lr_"+num_to_str(LR2)+"_sch_decay_"+num_to_str(SCHEDULE_DECAY2)+"_batch_size_"+str(BATCH_SIZE2)

print hyppar
print("DROPOUT", DROPOUT)
print("LR", LR)
print("SCHEDULE_DECAY", SCHEDULE_DECAY)

print hyppar2
print("DROPOUT2", DROPOUT2)
print("LR2", LR2)
print("SCHEDULE_DECAY2", SCHEDULE_DECAY2)


features = trainvar # = BDTvariables
features2 = BDTvariables_wo_gen_mHH

def nn_model_binary():
    "create a model."
    model = Sequential()
    model.add(Dense(2*len(features), input_dim=len(features), kernel_initializer='he_uniform')) 
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Dropout(DROPOUT)) #0.1
    for Nnodes in [8,8] :
        model.add(Dense(Nnodes, kernel_initializer='he_uniform'))
        model.add(BatchNormalization())
        model.add(PReLU())
        model.add(Dropout(DROPOUT)) #0.1
    model.add(Dense(nclasses, activation='sigmoid'))
    model.compile(
    loss='binary_crossentropy', 
    optimizer=Nadam(lr=LR, schedule_decay=SCHEDULE_DECAY), ##  lr=0.0005, sch_dec.=0.00005   # , beta_1 = 0.95, beta_2 = 0.999
    metrics=['accuracy'], 
    )
    return model

def nn_model_binary_wo_gen_mHH():
    "create a model."
    model = Sequential()
    model.add(Dense(2*len(features2), input_dim=len(features2), kernel_initializer='he_uniform')) 
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Dropout(DROPOUT2)) #0.1
    for Nnodes in [8,8] :
        model.add(Dense(Nnodes, kernel_initializer='he_uniform'))
        model.add(BatchNormalization())
        model.add(PReLU())
        model.add(Dropout(DROPOUT2)) #0.1
    model.add(Dense(nclasses, activation='sigmoid'))
    model.compile(
    loss='binary_crossentropy', 
    optimizer=Nadam(lr=LR2, schedule_decay=SCHEDULE_DECAY2), ##  lr=0.0005, sch_dec.=0.00005   # , beta_1 = 0.95, beta_2 = 0.999
    metrics=['accuracy'], 
    )
    return model

In [ ]:
nn_model_binary().summary()
nn_model_binary_wo_gen_mHH().summary()

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
# Port Keras Framework into SK-Learn
# https://stackoverflow.com/questions/39467496/error-when-using-keras-sk-learn-api
k_model_binary  = KerasClassifier(
    build_fn=nn_model_binary, ## For gen_mHH as input Var.
    epochs=EPOCHS,            ## For gen_mHH as input Var.
    batch_size=BATCH_SIZE,    ## For gen_mHH as input Var.
    verbose=2                 ## For gen_mHH as input Var.
    #build_fn=nn_model_binary_wo_gen_mHH, ## w/o gen_mHH as input Var.
    #epochs=EPOCHS2,                      ## w/o gen_mHH as input Var. 
    #batch_size=BATCH_SIZE2,              ## w/o gen_mHH as input Var.
    #verbose=2                            ## w/o gen_mHH as input Var.
)

k_model_binary2  = KerasClassifier(
    build_fn=nn_model_binary, ## For gen_mHH as input Var.
    epochs=EPOCHS,            ## For gen_mHH as input Var. 
    batch_size=BATCH_SIZE,    ## For gen_mHH as input Var.
    verbose=2                 ## For gen_mHH as input Var.
    #build_fn=nn_model_binary_wo_gen_mHH, ## w/o gen_mHH as input Var.
    #epochs=EPOCHS2,                      ## w/o gen_mHH as input Var. 
    #batch_size=BATCH_SIZE2,              ## w/o gen_mHH as input Var.
    #verbose=2                            ## w/o gen_mHH as input Var.


)

print("data_odd[features].values", data_odd[features].values)
print("data_odd[target].values", data_odd['target'].values)
print("sample_weight=data_odd[weight_train].values", data_odd["weight_train"].values)

history = k_model_binary.fit(
    data_odd[features].values, ## For gen_mHH as input Var.
    #data_odd[features2].values, ## w/o gen_mHH as input Var.
    data_odd['target'].values,
    sample_weight=data_odd["weight_train"].values,
    validation_data=(
        data_even[features].values, ## For gen_mHH as input Var.
        #data_even[features2].values, ## w/o gen_mHH as input Var.
        data_even['target'].values, 
        data_even["weight_train"].values
    )
)


history2 = k_model_binary2.fit(
    data_even[features].values, ## For gen_mHH as input Var.
    #data_even[features2].values, ## w/o gen_mHH as input Var.
    data_even['target'].values,
    sample_weight=data_even["weight_train"].values,
    validation_data=(
        data_odd[features].values, ## For gen_mHH as input Var.
        #data_odd[features2].values,  ## w/o gen_mHH as input Var. 
        data_odd['target'].values, 
        data_odd["weight_train"].values
    )
)


In [ ]:
"""
overtraining test
"""
# Extract number of run epochs from the training history
epochs = range(1, len(history.history["loss"])+1)
plt.figure(figsize=(9, 4))
#fig = plt.figure(figsize=(4, 4))
#plt.subplot(1, 2, 1)
plt.subplot(2, 2, 1)
# Extract loss on training and validation dataset and plot them together
plt.plot(epochs, history.history["loss"], "o-", label="Training (Odd)")
plt.plot(epochs, history.history["val_loss"], "o-", label="Validation (Even)")
plt.xlabel("Epochs"), plt.ylabel("Loss")
plt.yscale("log")
#plt.xlim(0,40)
plt.ylim(0.2,0.8)
plt.grid()
plt.legend(loc="best");

#plt.subplot(1, 2, 2)
plt.subplot(2, 2, 2)
#fig = plt.figure(figsize=(4, 4))
# Extract loss on training and validation dataset and plot them together
plt.plot(epochs, history.history["acc"], "o-", label="Training (Odd)")
plt.plot(epochs, history.history["val_acc"], "o-", label="Validation (Even)")
plt.xlabel("Epochs"), plt.ylabel("Accuracy")
plt.yscale("log")
plt.ylim(0.4,0.9)
plt.grid()
plt.legend(loc="best")

plt.subplot(2, 2, 3)
# Extract loss on training and validation dataset and plot them together
plt.plot(epochs, history2.history["loss"], "o-", label="Training (Even)")
plt.plot(epochs, history2.history["val_loss"], "o-", label="Validation (Odd)")
plt.xlabel("Epochs"), plt.ylabel("Loss")
plt.yscale("log")
#plt.xlim(0,40)
plt.ylim(0.2,0.8)
plt.grid()
plt.legend(loc="best");

plt.subplot(2, 2, 4)
#fig = plt.figure(figsize=(4, 4))
# Extract loss on training and validation dataset and plot them together
plt.plot(epochs, history2.history["acc"], "o-", label="Training (Even)")
plt.plot(epochs, history2.history["val_acc"], "o-", label="Validation (Odd)")
plt.xlabel("Epochs"), plt.ylabel("Accuracy")
plt.yscale("log")
plt.ylim(0.4,0.9)
plt.grid()
plt.legend(loc="best")

filename1="{}/loss_and_acc_{}_Log.pdf".format(channel, hyppar) ## For gen_mHH as input Var.
#filename1="{}/loss_and_acc_{}_Log.pdf".format(channel, hyppar2) ## w/o gen_mHH as input Var.
plt.savefig(filename1);

plt.show()

In [ ]:
import sys
sys.path.append('/home/ram/.local/lib/python2.7/site-packages')
import eli5
from eli5.sklearn import PermutationImportance
"""
to calculate variables importance, it takes time and it is not completelly 'enlightant',
do not do all the time.
"""

In [ ]:
print ("Date: ", time.asctime( time.localtime(time.time()) ))
perm = PermutationImportance(k_model_binary, random_state=1).fit( # , scoring="f1_samples"
    data_odd[features].values, ## For gen_mHH as input Var.
    #data_odd[features2].values, ## w/o gen_mHH as input Var.
    data_odd['target'].values,
    sample_weight=data_odd["weight_train"].values
)
print ("Date: ", time.asctime( time.localtime(time.time()) ))
eli5.show_weights(perm, feature_names = data_odd[features].columns.tolist(), top=len(features)) ## For gen_mHH as input Var.
#eli5.show_weights(perm, feature_names = data_odd[features2].columns.tolist(), top=len(features2)) ## w/o gen_mHH as input Var.

print ("Date: ", time.asctime( time.localtime(time.time()) ))
perm2 = PermutationImportance(k_model_binary2, random_state=1).fit( # , scoring="f1_samples"
    data_even[features].values, ## For gen_mHH as input Var.
    #data_even[features2].values, ## w/o gen_mHH as input Var.
    data_even['target'].values,
    sample_weight=data_even["weight_train"].values
)
print ("Date: ", time.asctime( time.localtime(time.time()) ))
eli5.show_weights(perm2, feature_names = data_even[features].columns.tolist(), top=len(features)) ## For gen_mHH as input Var.
#eli5.show_weights(perm2, feature_names = data_even[features2].columns.tolist(), top=len(features2)) ## w/o gen_mHH as input Var.

In [ ]:
"""
Calculate the output in all dataset 
-- to pass to the training/test
"""

## For gen_mHH as input Var.
data_odd["NN_output"]  = k_model_binary.predict_proba(data_odd[features].values, verbose=1)[:, 1]
data_even["NN_output"] = k_model_binary2.predict_proba(data_even[features].values, verbose=1)[:, 1]

## w/o gen_mHH as input Var.
#data_odd["NN_output"]  = k_model_binary.predict_proba(data_odd[features2].values, verbose=1)[:, 1]
#data_even["NN_output"] = k_model_binary2.predict_proba(data_even[features2].values, verbose=1)[:, 1]



In [ ]:
hist_params = {'normed': True, 'bins': 10 , 'histtype':'step'}
target = 'target'
plt.clf()

plt.figure('XGB',figsize=(6, 6))

values, bins, _ = plt.hist(
    data_odd.loc[data_odd.target.values == 1, "NN_output"].values , 
    weights=data_odd.loc[data_odd.target.values == 1, "weight_cx"].values,
    label="sig odd", color='r', range=(0,1), **hist_params
    )
values, bins, _ = plt.hist(
    data_odd.loc[data_odd.target.values == 0, "NN_output"].values , 
    weights=data_odd.loc[data_odd.target.values == 0, "weight_cx"].values,
    label="BKG odd", color='g', range=(0,1), **hist_params
    )

values, bins, _ = plt.hist(
    data_even.loc[data_even.target.values == 1, "NN_output"].values , 
    weights=(data_even.loc[data_even.target.values == 1, "weight_cx"].values),
    label="sig even", color='r', ls='--', range=(0,1), **hist_params)
values, bins, _ = plt.hist(
    data_even.loc[data_even.target.values == 0, "NN_output"].values , 
    weights=(data_even.loc[data_even.target.values == 0, "weight_cx"].values),
    label="BKG even", color='g', ls='--', range=(0,1), **hist_params)

#plt.xscale('log')
plt.yscale('log')
plt.legend(loc='upper center')
#filename2="{}/NN_Output_{}.pdf".format(channel, hyppar) ## For gen_mHH as input Var.
filename2="{}/NN_Output_{}.pdf".format(channel, hyppar2) ## w/o gen_mHH as input Var.
plt.savefig(filename2);

plt.show()



In [ ]:
###############################
# ROC Curve
styleline = ['-', '--', '-.', ':']
colors_mass = ['m', 'b', 'k', 'r', 'g',  'y', 'c', ]
fig, ax = plt.subplots(figsize=(6, 6))
sl = 0

for dd, data_do in  enumerate(order_train) :
        if dd == 0 : val_data = 1
        else : val_data = 0
        #print data_do["weight_cx"].astype(np.float64)  
        #print data_do.NN_output.values
        fpr, tpr, thresholds = roc_curve(
                data_do["target"].astype(np.bool), 
                data_do.NN_output.values, 
                sample_weight=data_do["weight_cx"].astype(np.float64)
        )
        train_auc = auc(fpr, tpr, reorder = True)
        print("train set auc " + str(train_auc))
        fprt, tprt, thresholds = roc_curve(
            order_train[val_data]["target"].astype(np.bool), 
            order_train[val_data].NN_output.values, 
            sample_weight=(order_train[val_data]["weight_cx"].astype(np.float64))
        )
        test_auct = auc(fprt, tprt, reorder = True)
        print("test set auc " + str(test_auct))
        ax.plot(
            fpr, tpr,
            lw = 2, linestyle = styleline[dd + dd*1], color = colors_mass[1],
            label = order_train_name[dd] + ' train (area = %0.3f)'%(train_auc) + ")"
            )
        sl += 1
        ax.plot(
            fprt, tprt,
            lw = 2, linestyle = styleline[dd + 1 + + dd*1], color = colors_mass[2],
            label = order_train_name[dd] + ' test (area = %0.3f)'%(test_auct) + ")"
            )
        sl += 1
ax.set_ylim([0.0,1.0])
ax.set_xlim([0.0,1.0])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.legend(loc="lower right", fontsize = 'small')
ax.grid()

#filename2a="{}/ROC_{}.pdf".format(channel, hyppar) ## For gen_mHH as input Var.
filename2a="{}/ROC_{}.pdf".format(channel, hyppar2) ## w/o gen_mHH as input Var.
plt.savefig(filename2a);


In [ ]:
###############################
## classifier plot by mass
hist_params = {'normed': True, 'bins': 8 , 'histtype':'step', "lw": 2}
plt.clf()
colorcold = ['g', 'r', 'y']
colorhot = ['b', 'magenta', 'orange']

fig, ax = plt.subplots(figsize=(6, 6))
for mm, mass in enumerate(output["masses_test"]) :
    y_pred = data_even.loc[(data_even.target.values == 0) & (data_even["gen_mHH"] == mass), "NN_output"].values
    y_predS = data_even.loc[(data_even.target.values == 1) & (data_even["gen_mHH"] == mass), "NN_output"].values
    y_pred_train = data_odd.loc[(data_odd.target.values == 0) & (data_odd["gen_mHH"] == mass), "NN_output"].values
    y_predS_train = data_odd.loc[(data_odd.target.values == 1) & (data_odd["gen_mHH"] == mass), "NN_output"].values
    dict_plot = [
       [y_pred, "-", colorhot[mm],  str(mass)+" GeV test BKG"],
       [y_predS, "-", colorcold[mm], str(mass)+" GeV test signal"],
       [y_pred_train, "--", colorhot[mm], str(mass)+" GeV train BKG" ],
       [y_predS_train, "--", colorcold[mm],      str(mass)+" GeV train signal"]
    ]
    for item in dict_plot :
        values1, bins, _ = ax.hist(
            item[0],
            ls=item[1], color = item[2],
            label=item[3],
            range=(0,1),
            **hist_params
            )
        normed = sum(y_pred)
        mid = 0.5*(bins[1:] + bins[:-1])
        err=np.sqrt(values1*normed)/normed # denominator is because plot is normalized
        plt.errorbar(mid, values1, yerr=err, fmt='none', color= item[2], ecolor= item[2], edgecolor=item[2], lw=2)
#plt.xscale('log')
#plt.yscale('log')
ax.legend(loc='upper center', title="by mass ", fontsize = 'small')
#nameout = channel+'/'+bdtType+'_'+trainvar+'_'+str(len(trainVars(False)))+'_'+hyppar+'_mass_'+ str(mass)+'_XGBclassifier.pdf'

filename3="{}/NN_Output_by_mass_{}.pdf".format(channel, hyppar) ## For gen_mHH as input Var.
#filename3="{}/NN_Output_by_mass_{}.pdf".format(channel, hyppar2) ## w/o gen_mHH as input Var.
plt.savefig(filename3);

In [ ]:
###############################
# by mass ROC
styleline = ['-', '--', '-.', ':']
colors_mass = ['m', 'b', 'k', 'r', 'g',  'y', 'c', ]
fig, ax = plt.subplots(figsize=(6, 6))
sl = 0


for mm, mass in enumerate(output["masses_test"]) :
    for dd, data_do in  enumerate(order_train) :
        if dd == 0 : val_data = 1
        else : val_data = 0
        fpr, tpr, thresholds = roc_curve(
            data_do.loc[(data_do["gen_mHH"] == mass), "target"].astype(np.bool),
            data_do.loc[(data_do["gen_mHH"] == mass), "NN_output"].values,
            sample_weight=(data_do.loc[(data_do["gen_mHH"].astype(np.int) == int(mass)), "weight_cx"].astype(np.float64))
        )
        train_auc = auc(fpr, tpr, reorder = True)
        print("train set auc " + str(train_auc) + " (mass = " + str(mass) + ")")
        fprt, tprt, thresholds = roc_curve(
            order_train[val_data].loc[(order_train[val_data]["gen_mHH"].astype(np.int) == int(mass)), target].astype(np.bool), 
            order_train[val_data].loc[(order_train[val_data]["gen_mHH"] == mass), "NN_output"].values, #proba[:,1],
            sample_weight=(order_train[val_data].loc[(order_train[val_data]["gen_mHH"].astype(np.int) == int(mass)), "weight_cx"].astype(np.float64))
        )
        test_auct = auc(fprt, tprt, reorder = True)
        print("test set auc " + str(test_auct) + " (mass = " + str(mass) + ")")
        ax.plot(
            fpr, tpr,
            lw = 2, linestyle = styleline[dd + dd*1], color = colors_mass[mm],
            label = order_train_name[dd] + ' train (area = %0.3f)'%(train_auc) + " (mass = " + str(mass) + ")"
            )
        sl += 1
        ax.plot(
            fprt, tprt,
            lw = 2, linestyle = styleline[dd + 1 + + dd*1], color = colors_mass[mm],
            label = order_train_name[dd] + ' test (area = %0.3f)'%(test_auct) + " (mass = " + str(mass) + ")"
            )
        sl += 1
ax.set_ylim([0.0,1.0])
ax.set_xlim([0.0,1.0])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.legend(loc="lower right", fontsize = 'small')
ax.grid()

filename4="{}/ROC_by_mass_{}.pdf".format(channel, hyppar) ## For gen_mHH as input Var.
#filename4="{}/ROC_by_mass_{}.pdf".format(channel, hyppar2) ## w/o gen_mHH as input Var.
plt.savefig(filename4);


In [ ]:
import matplotlib.pyplot as plt2

features = trainvar # = BDTvariables
features2 = BDTvariables_wo_gen_mHH

#### ----- Interpolation check log files ----####                                                                                                                                                                                                                       
styleline = ['-', '--', '-.', ':', 'solid']
colors_mass = ['m', 'b', 'k', 'r', 'g',  'y', 'c', ]
## ---- ROC CURVES ------###
#fig1, ax1 = plt2.subplots(figsize=(6, 6))
#fig2, ax2 = plt2.subplots(figsize=(6, 6))
#fig3, ax3 = plt2.subplots(figsize=(6, 6))
#sl = 0
#s2 = 0
#s3 = 0

hist_params = {'normed': True, 'bins': 8 , 'histtype':'step', "lw": 2}
plt.clf()
colorcold = ['g', 'r', 'y', 'black', 'gray']
colorhot = ['b', 'magenta', 'orange', 'cyan', 'brown']
## ---- NN OUTPUT ---- ###
fig1a, ax1a = plt.subplots(figsize=(6, 6))
fig1b, ax1b = plt.subplots(figsize=(6, 6))
fig1c, ax1c = plt.subplots(figsize=(6, 6))

for mm, mass in enumerate(test_masses): ## Loop over the test masses   
#for mm, mass in enumerate([300]): ## Loop over 300 GeV test mass only      
    print("mass", mass)
    MASS = int(mass)
    print("MASS", MASS)
    MASS_STR = "{}".format(MASS)
    print("MASS_STR", MASS_STR)
    
    ## For writing the interpolation check roc curves ##
    log_file2_name = "{}/{}_{}.log".format(channel,channel,MASS_STR)
    file2_ = open(log_file2_name, 'w+')

    for dd, data_do in  enumerate(order_train):   ## Loop over the odd and even dataset halves                                                                                                                                                                          
        if dd == 0 : val_data = 1
        else : val_data = 0
        
        if(Bkg_mass_rand == "default"):
            print("Using the new logic")
            traindataset1 = data_do.loc[~((data_do["gen_mHH"]==mass) & (data_do['target']==1))]  ## Training for all masses except the one under study only for signal process                                                                                            
            #valdataset1   = order_train[val_data].loc[~((order_train[val_data]["gen_mHH"]==mass) & (order_train[val_data][target]==1))]  ## Testing for all masses except the one under study for signal process                                                       
            valdataset1 = order_train[val_data].loc[~((order_train[val_data]["gen_mHH"]!=mass) & (order_train[val_data]['target']==1))] ## Testing on only the mass under study for signal                                                                                
        else:
            print("Using the old logic")
            traindataset1 = data_do.loc[~(data_do["gen_mHH"]==mass)]  ## Training for all masses except the one under study                                                                                                                                             
            #valdataset1 = order_train[val_data].loc[~(order_train[val_data]["gen_mHH"]==mass)]  ## Testing for all masses except the one under study                                                                                                                   
            valdataset1 = order_train[val_data].loc[(order_train[val_data]["gen_mHH"]==mass)] ## Testing on only the mass under study
        
        
        if(mass == 300):
            if(TrainMode == 0):
                masses1 = [250,260,270,280,350,400,450,500,550,600,650,700,750,800,850,900,1000] ## All masses except 300                                                                                                                                               
                masses_test1 = [250,1000]
            elif(TrainMode == 1):
                masses1 = [250,260,270,280,350,400] ## Low mass only training (except 300)                                                                                                                                                                              
                masses_test1 = [250]
            masses2 = [300]
            masses_test2 = [300]
        elif(mass == 500):
            if(TrainMode == 0):
                masses1 = [250,260,270,280,300,350,400,450,550,600,650,700,750,800,850,900,1000] ## All masses except 500                                                                                                                                               
                masses_test1 = [250,1000]
            elif(TrainMode == 2):
                masses1 = [450,550,600,650,700,750,800,850,900,1000] ## High mass only training (except 500)                                                                                                                                                            
                masses_test1 = [1000]
            masses2 = [500]
            masses_test2 = [500]
        elif(mass == 800):
            if(TrainMode == 0):
                masses1 = [250,260,270,280,300,350,400,450,500,550,600,650,700,750,850,900,1000] ## All masses except 800                                                                                                                                               
                masses_test1 = [250,1000]
            elif(TrainMode == 2):
                masses1 = [450,500,550,600,650,700,750,850,900,1000] ## High mass only training (except 800)                                                                                                                                                            
                masses_test1 = [1000]
            masses2 = [800]
            masses_test2 = [800]
           
        
        ################# WITH gen_mHH #####################                                                                                                                                                                                                            
        k_model_binary2  = KerasClassifier(
                       build_fn=nn_model_binary, 
                       epochs=EPOCHS, # 10
                       batch_size=BATCH_SIZE, # 64
                       verbose=2
                       )
        
        history2 = k_model_binary2.fit(
               traindataset1[features].values, 
               traindataset1['target'].values,
               sample_weight=traindataset1["weight_train"].values,
               validation_data=(
                   valdataset1[features].values, 
                   valdataset1['target'].values, 
                   valdataset1["weight_train"].values
               )
        )

        traindataset1["NN_output2"] = 1.0
        valdataset1["NN_output2"]   = 1.0
        
        ## --- ROC curve ----##                                                                                                                                                                                                                                         
        #proba = cls2.predict_proba(valdataset1[trainVars(False, options.variables, options.bdtType)].values )
        valdataset1["NN_output2"] = k_model_binary2.predict_proba(valdataset1[features].values, verbose=1)[:, 1]
        fprt, tprt, thresholds = roc_curve(valdataset1['target'].astype(np.bool), valdataset1["NN_output2"].values, sample_weight=(valdataset1["weight_cx"].astype(np.float64)) )
        test_auct = auc(fprt, tprt, reorder = True)

        #proba = cls2.predict_proba(traindataset1[trainVars(False, options.variables, options.bdtType)].values )
        traindataset1["NN_output2"] = k_model_binary2.predict_proba(traindataset1[features].values, verbose=1)[:, 1]
        fpr, tpr, thresholds = roc_curve(traindataset1['target'].astype(np.bool), traindataset1["NN_output2"].values, sample_weight=(traindataset1["weight_cx"].astype(np.float64)) )
        train_auc = auc(fpr, tpr, reorder = True)

        ## ---- BDT Output Distributions ----- ###                                                                                                                                                                                                                      
        y_pred = k_model_binary2.predict_proba(valdataset1.ix[valdataset1.target.values == 0, BDTvariables].values)[:, 1]
        y_predS = k_model_binary2.predict_proba(valdataset1.ix[valdataset1.target.values == 1, BDTvariables].values)[:, 1]
        
        y_pred_train = k_model_binary2.predict_proba(traindataset1.ix[traindataset1.target.values == 0, BDTvariables].values)[:, 1]
        y_predS_train = k_model_binary2.predict_proba(traindataset1.ix[traindataset1.target.values == 1, BDTvariables].values)[:, 1]

        file2_.write('#------ Training for all masses except %0.1f GeV-------\n' %mass)
        file2_.write('#------ Testing only for mass %0.1f GeV (using other dataset half)--------\n' %mass)
        if(dd == 0): ## Train->odd; Test/val -> even                                                                                                                                                                                                                    
            file2_.write('train_auc_ODD0_%s = %0.8f\n' % (MASS_STR, train_auc))
            file2_.write('test_auc_EVEN0_%s  = %0.8f\n' % (MASS_STR, test_auct))
            file2_.write('xtrain_ODD0_%s = ' % (MASS_STR))
            file2_.write(str(fpr.tolist()))
            file2_.write('\n')
            file2_.write('ytrain_ODD0_%s = ' % (MASS_STR))
            file2_.write(str(tpr.tolist()))
            file2_.write('\n')
            file2_.write('xval_EVEN0_%s = ' % (MASS_STR))
            file2_.write(str(fprt.tolist()))
            file2_.write('\n')
            file2_.write('yval_EVEN0_%s = ' % (MASS_STR))
            file2_.write(str(tprt.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_test_EVEN0_%s = ' % (MASS_STR))
            file2_.write(str(y_pred.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_test_EVEN0_%s = ' % (MASS_STR))
            file2_.write(str(y_predS.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_train_ODD0_%s = ' % (MASS_STR))
            file2_.write(str(y_pred_train.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_train_ODD0_%s = ' % (MASS_STR))
            file2_.write(str(y_predS_train.tolist()))
            file2_.write('\n')
        else: ## Train->even; Test/val -> odd                                                                                                                                                                                                                           
            file2_.write('train_auc_EVEN0_%s = %0.8f\n' %  (MASS_STR, train_auc))
            file2_.write('test_auc_ODD0_%s  = %0.8f\n' %  (MASS_STR, test_auct))
            file2_.write('xtrain_EVEN0_%s = ' % (MASS_STR))
            file2_.write(str(fpr.tolist()))
            file2_.write('\n')
            file2_.write('ytrain_EVEN0_%s = ' % (MASS_STR))
            file2_.write(str(tpr.tolist()))
            file2_.write('\n')
            file2_.write('xval_ODD0_%s = ' % (MASS_STR))
            file2_.write(str(fprt.tolist()))
            file2_.write('\n')
            file2_.write('yval_ODD0_%s = ' % (MASS_STR))
            file2_.write(str(tprt.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_test_ODD0_%s = ' % (MASS_STR))
            file2_.write(str(y_pred.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_test_ODD0_%s = ' % (MASS_STR))
            file2_.write(str(y_predS.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_train_EVEN0_%s = ' % (MASS_STR))
            file2_.write(str(y_pred_train.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_train_EVEN0_%s = ' % (MASS_STR))
            file2_.write(str(y_predS_train.tolist()))
            file2_.write('\n')
        file2_.write('##-------------------------------------------##\n')
        ################################################### 
        
        ## ---- ROC curves ---###
        #ax1.plot( 
        #    fpr, tpr,
        #    lw = 2, linestyle = styleline[dd + dd*1], color = colors_mass[0],
        #    label = order_train_name[dd] + ' train all except %s GeV w gen_mHH (area = %0.3f)'%(str(mass), train_auc)
        #    )
        #sl += 1
        #ax1.plot(
        #    fprt, tprt,
        #    lw = 2, linestyle = styleline[dd + 1 + + dd*1], color = colors_mass[0],
        #    label = order_train_name[dd] + ' test only %s GeV w gen_mHH (area = %0.3f)'%(str(mass), test_auct)
        #    )
        #sl += 1
        
        ## Plot NN Output Histograms
        if dd == 0 : # Odd train, Even test
            dict_plot = [ ## Output Histogtams
                   [y_pred, "-", colorhot[mm],  str(mass)+" GeV test BKG (Even)"],
                   [y_predS, "-", colorcold[mm], str(mass)+" GeV test signal (Even)"],
                   [y_pred_train, "--", colorhot[mm], str(mass)+" GeV train BKG (Odd)" ],
                   [y_predS_train, "--", colorcold[mm],      str(mass)+" GeV train signal (Odd)"]
                ]
        else: # Even train, Odd test
            dict_plot = [ ## Output Histogtams
                   [y_pred, "-", colorhot[mm+1],  str(mass)+" GeV test BKG (Odd)"],
                   [y_predS, "-", colorcold[mm+1], str(mass)+" GeV test signal (Odd)"],
                   [y_pred_train, "--", colorhot[mm+1], str(mass)+" GeV train BKG (Even)" ],
                   [y_predS_train, "--", colorcold[mm+1],      str(mass)+" GeV train signal (Even)"]
                ]
            
        for item in dict_plot :
            values1, bins, _ = ax1a.hist(
                item[0],
                ls=item[1], color = item[2],
                label=item[3],
                range=(0,1),
                **hist_params
                )
            normed = sum(y_pred)
            mid = 0.5*(bins[1:] + bins[:-1])
            err=np.sqrt(values1*normed)/normed # denominator is because plot is normalized
            plt.errorbar(mid, values1, yerr=err, fmt='none', color= item[2], ecolor= item[2], edgecolor=item[2], lw=2)
        del dict_plot[:] ## Clearing the list    
        
        ################# W/O gen_mHH #####################                                                                                                                                                                                                             
        k_model_binary2a = KerasClassifier(
                       build_fn=nn_model_binary_wo_gen_mHH, 
                       epochs=EPOCHS2, # 10
                       batch_size=BATCH_SIZE2, # 64
                       verbose=2
                       )

        history2a = k_model_binary2a.fit(
               traindataset1[BDTvariables_wo_gen_mHH].values, 
               traindataset1['target'].values,
               sample_weight=traindataset1["weight_train"].values,
               validation_data=(
                   valdataset1[BDTvariables_wo_gen_mHH].values, 
                   valdataset1['target'].values, 
                   valdataset1["weight_train"].values
               )
        )

        traindataset1["NN_output2a"] = 1.0
        valdataset1["NN_output2a"]   = 1.0
        
        ## --- ROC curve ----##                                                                                                                                                                                                                                         
        #proba_a = cls2a.predict_proba(valdataset1[BDTvariables_wo_gen_mHH].values )
        valdataset1["NN_output2a"] = k_model_binary2a.predict_proba(valdataset1[BDTvariables_wo_gen_mHH].values , verbose=1)[:, 1]
        fprt_a, tprt_a, thresholds = roc_curve(valdataset1['target'].astype(np.bool), valdataset1["NN_output2a"].values, sample_weight=(valdataset1["weight_cx"].astype(np.float64))  )
        test_auct_a = auc(fprt_a, tprt_a, reorder = True)

        #proba_a = cls2a.predict_proba(traindataset1[BDTvariables_wo_gen_mHH].values )
        traindataset1["NN_output2a"] = k_model_binary2a.predict_proba(traindataset1[BDTvariables_wo_gen_mHH].values , verbose=1)[:, 1]
        fpr_a, tpr_a, thresholds = roc_curve(traindataset1['target'].astype(np.bool), traindataset1["NN_output2a"].values, sample_weight=(traindataset1["weight_cx"].astype(np.float64)) )
        train_auc_a = auc(fpr_a, tpr_a, reorder = True)

        ## ---- BDT Output Distributions ----- ###                                                                                                                                                                                                                      
        y_pred_a = k_model_binary2a.predict_proba(valdataset1.ix[valdataset1.target.values == 0, BDTvariables_wo_gen_mHH].values)[:, 1]
        y_predS_a = k_model_binary2a.predict_proba(valdataset1.ix[valdataset1.target.values == 1, BDTvariables_wo_gen_mHH].values)[:, 1]

        y_pred_train_a = k_model_binary2a.predict_proba(traindataset1.ix[traindataset1.target.values == 0, BDTvariables_wo_gen_mHH].values)[:, 1]
        y_predS_train_a = k_model_binary2a.predict_proba(traindataset1.ix[traindataset1.target.values == 1, BDTvariables_wo_gen_mHH].values)[:, 1]
        
        file2_.write('#------ Training for all masses except %0.1f GeV (w/o gen_mHH)-------\n' %mass)
        file2_.write('#------ Testing only for mass %0.1f GeV using other dataset half (w/o gen_mHH)--------\n' %mass)
        if(dd == 0): ## Train->odd; Test/val -> even                                                                                                                                                                                                                    
            file2_.write('train_auc_ODD0a_%s = %0.8f\n' % (MASS_STR, train_auc_a))
            file2_.write('test_auc_EVEN0a_%s  = %0.8f\n' % (MASS_STR, test_auct_a))
            file2_.write('xtrain_ODD0a_%s = ' % (MASS_STR))
            file2_.write(str(fpr_a.tolist()))
            file2_.write('\n')
            file2_.write('ytrain_ODD0a_%s = ' % (MASS_STR))
            file2_.write(str(tpr_a.tolist()))
            file2_.write('\n')
            file2_.write('xval_EVEN0a_%s = ' % (MASS_STR))
            file2_.write(str(fprt_a.tolist()))
            file2_.write('\n')
            file2_.write('yval_EVEN0a_%s = ' % (MASS_STR))
            file2_.write(str(tprt_a.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_test_EVEN0a_%s = ' % (MASS_STR))
            file2_.write(str(y_pred_a.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_test_EVEN0a_%s = ' % (MASS_STR))
            file2_.write(str(y_predS_a.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_train_ODD0a_%s = ' % (MASS_STR))
            file2_.write(str(y_pred_train_a.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_train_ODD0a_%s = ' % (MASS_STR))
            file2_.write(str(y_predS_train_a.tolist()))
            file2_.write('\n')
        else: ## Train->even; Test/val -> odd                                                                                                                                                                                                                           
            file2_.write('train_auc_EVEN0a_%s = %0.8f\n' %  (MASS_STR, train_auc_a))
            file2_.write('test_auc_ODD0a_%s  = %0.8f\n' %  (MASS_STR, test_auct_a))
            file2_.write('xtrain_EVEN0a_%s = ' % (MASS_STR))
            file2_.write(str(fpr_a.tolist()))
            file2_.write('\n')
            file2_.write('ytrain_EVEN0a_%s = ' % (MASS_STR))
            file2_.write(str(tpr_a.tolist()))
            file2_.write('\n')
            file2_.write('xval_ODD0a_%s = ' % (MASS_STR))
            file2_.write(str(fprt_a.tolist()))
            file2_.write('\n')
            file2_.write('yval_ODD0a_%s = ' % (MASS_STR))
            file2_.write(str(tprt_a.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_test_ODD0a_%s = ' % (MASS_STR))
            file2_.write(str(y_pred_a.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_test_ODD0a_%s = ' % (MASS_STR))
            file2_.write(str(y_predS_a.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_train_EVEN0a_%s = ' % (MASS_STR))
            file2_.write(str(y_pred_train_a.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_train_EVEN0a_%s = ' % (MASS_STR))
            file2_.write(str(y_predS_train_a.tolist()))
            file2_.write('\n')
        file2_.write('##-------------------------------------------##\n')
        ################################################### 
        
        ## --- ROC Curves --- ##
        #ax2.plot(
        #    fpr_a, tpr_a,
        #    lw = 2, linestyle = styleline[dd + dd*1], color = colors_mass[1],
        #    label = order_train_name[dd] + ' train all except %s GeV w/o gen_mHH (area = %0.3f)'%(str(mass), train_auc_a)
        #    )
        #s2 += 1
        #ax2.plot(
        #    fprt_a, tprt_a,
        #    lw = 2, linestyle = styleline[dd + 1 + + dd*1], color = colors_mass[1],
        #    label = order_train_name[dd] + ' test only %s GeV w/o gen_mHH (area = %0.3f)'%(str(mass), test_auct_a)
        #    )
        #s2 += 1
        
        ## Plot NN Output Histograms
        if dd == 0 : # Odd train, Even test
            dict_plot2 = [ ## Output Histogtams
                   [y_pred_a, "-", colorhot[mm],  str(mass)+" GeV test BKG (Even)"],
                   [y_predS_a, "-", colorcold[mm], str(mass)+" GeV test signal (Even)"],
                   [y_pred_train_a, "--", colorhot[mm], str(mass)+" GeV train BKG (Odd)" ],
                   [y_predS_train_a, "--", colorcold[mm],      str(mass)+" GeV train signal (Odd)"]
                ]
        else: # Even train, Odd test
            dict_plot2 = [ ## Output Histogtams
                   [y_pred_a, "-", colorhot[mm+1],  str(mass)+" GeV test BKG (Odd)"],
                   [y_predS_a, "-", colorcold[mm+1], str(mass)+" GeV test signal (Odd)"],
                   [y_pred_train_a, "--", colorhot[mm+1], str(mass)+" GeV train BKG (Even)" ],
                   [y_predS_train_a, "--", colorcold[mm+1],      str(mass)+" GeV train signal (Even)"]
                ]
            
        for item in dict_plot2 :
            values1, bins, _ = ax1b.hist(
                item[0],
                ls=item[1], color = item[2],
                label=item[3],
                range=(0,1),
                **hist_params
                )
            normed = sum(y_pred_a)
            mid = 0.5*(bins[1:] + bins[:-1])
            err=np.sqrt(values1*normed)/normed # denominator is because plot is normalized
            plt.errorbar(mid, values1, yerr=err, fmt='none', color= item[2], ecolor= item[2], edgecolor=item[2], lw=2)
        del dict_plot2[:] ## Clearing the list 
            
        ## --- Single Mass training ----##                                                                                                                                                                                                                                         
        if(Bkg_mass_rand == "default"):
            print("Using the new logic")
            #traindataset2 = data_do.loc[(data_do["gen_mHH"]==mass)]  ## Training for only the mass point under study                                                                                                                                                   
            traindataset2 = data_do.loc[ ~((data_do["gen_mHH"]!=mass) & (data_do['target']==1)) ]  ## Training for only the mass point under study (for the signal process)                                                                                               
            valdataset2 = order_train[val_data].loc[ ~((order_train[val_data]["gen_mHH"]!=mass) & (order_train[val_data]['target']==1)) ]  ## Testing for only the mass point under study (for the signal process)                                                        
        else:
            print("Using the old logic")
            traindataset2 = data_do.loc[(data_do["gen_mHH"]==mass)]  ## Training for only the mass point under study                                                                                                                                                    
            valdataset2 = order_train[val_data].loc[(order_train[val_data]["gen_mHH"]==mass)]  ## Testing for only the mass point under study                                                                                                                           

        #traindataset2 = data_do.loc[(data_do["gen_mHH"]==mass)]  ## Training for only the mass point under study                                                                                                                                                       
        #valdataset2 = order_train[val_data].loc[(order_train[val_data]["gen_mHH"]==mass)]  ## Testing for only the mass point under study                                                                                                                              

       
        k_model_binary3 = KerasClassifier(
                       build_fn=nn_model_binary_wo_gen_mHH, 
                       epochs=EPOCHS2, # 10
                       batch_size=BATCH_SIZE2, # 64
                       verbose=2
                       #build_fn=nn_model_binary, 
                       #epochs=EPOCHS, # 10
                       #batch_size=BATCH_SIZE, # 64
                       #verbose=2
                       )
        
        history3 = k_model_binary3.fit(
               traindataset2[BDTvariables_wo_gen_mHH].values, ## Since we are doing single mass train no need for gen_mHH
               #traindataset2[BDTvariables].values,
               traindataset2['target'].values,
               sample_weight=traindataset2["weight_train"].values,
               validation_data=(
                   valdataset2[BDTvariables_wo_gen_mHH].values,
                   #valdataset2[BDTvariables].values,
                   valdataset2['target'].values, 
                   valdataset2["weight_train"].values
               )
        )
        
        traindataset2["NN_output3"] = 1.0
        valdataset2["NN_output3"]   = 1.0
        
        valdataset2["NN_output3"] = k_model_binary3.predict_proba(valdataset2[BDTvariables_wo_gen_mHH].values , verbose=1)[:, 1] ## Since we are doing single mass train no need for gen_mHH
        #valdataset2["NN_output3"] = k_model_binary3.predict_proba(valdataset2[BDTvariables].values )
        fprt3, tprt3, thresholds3 = roc_curve(valdataset2['target'].astype(np.bool), valdataset2["NN_output3"].values, sample_weight=(valdataset2["weight_cx"].astype(np.float64))  )
        print("fprt3", fprt3)
        print("tprt3", tprt3)
        test_auct = auc(fprt3, tprt3, reorder = True)

        traindataset2["NN_output3"] = k_model_binary3.predict_proba(traindataset2[BDTvariables_wo_gen_mHH].values , verbose=1)[:, 1] ## Since we are doing single mass train no need for gen_mHH                                                                                                                          
        #traindataset2["NN_output3"] = k_model_binary3.predict_proba(traindataset2[BDTvariables].values )
        fpr3, tpr3, thresholds3 = roc_curve(traindataset2['target'].astype(np.bool), traindataset2["NN_output3"].values, sample_weight=(traindataset2["weight_cx"].astype(np.float64)) )
        train_auc = auc(fpr3, tpr3, reorder = True)

        ## ---- BDT Output Distributions ----- ###                                                                                                                                                                                                                      
        y_pred_3 = k_model_binary3.predict_proba(valdataset2.ix[valdataset2.target.values == 0, BDTvariables_wo_gen_mHH].values)[:, 1]   ## Since we are doing single mass train no need for gen_mHH                                                                                 
        y_predS_3 = k_model_binary3.predict_proba(valdataset2.ix[valdataset2.target.values == 1, BDTvariables_wo_gen_mHH].values)[:, 1]  ## Since we are doing single mass train no need for gen_mHH                                                                                 

        y_pred_train_3 = k_model_binary3.predict_proba(traindataset2.ix[traindataset2.target.values == 0, BDTvariables_wo_gen_mHH].values)[:, 1]  ## Since we are doing single mass train no need for gen_mHH                                                                        
        y_predS_train_3 = k_model_binary3.predict_proba(traindataset2.ix[traindataset2.target.values == 1, BDTvariables_wo_gen_mHH].values)[:, 1] ## Since we are doing single mass train no need for gen_mHH 
  
        #y_pred_3 = k_model_binary3.predict_proba(valdataset2.ix[valdataset2.target.values == 0, BDTvariables].values)[:, 1]
        #y_predS_3 = k_model_binary3.predict_proba(valdataset2.ix[valdataset2.target.values == 1, BDTvariables].values)[:, 1]
    
        #y_pred_train_3 = k_model_binary3.predict_proba(traindataset2.ix[traindataset2.target.values == 0, BDTvariables].values)[:, 1]
        #y_predS_train_3 = k_model_binary3.predict_proba(traindataset2.ix[traindataset2.target.values == 1, BDTvariables].values)[:, 1] 
    
        file2_.write('#------ Training for only mass %0.1f GeV-------\n' %mass)
        file2_.write('#------ Testing for only mass %0.1f GeV (using other data half)--------\n' %mass)
        if(dd == 0): ## Train->odd; Test/val -> even                                                                                                                                                                                                                    
            file2_.write('train_auc_ODD1_%s = %0.8f\n' % (MASS_STR, train_auc))
            file2_.write('test_auc_EVEN1_%s  = %0.8f\n' % (MASS_STR, test_auct))
            file2_.write('xtrain_ODD1_%s = ' % (MASS_STR))
            file2_.write(str(fpr3.tolist()))
            file2_.write('\n')
            file2_.write('ytrain_ODD1_%s = ' % (MASS_STR))
            file2_.write(str(tpr3.tolist()))
            file2_.write('\n')
            file2_.write('xval_EVEN1_%s = ' % (MASS_STR))
            file2_.write(str(fprt3.tolist()))
            file2_.write('\n')
            file2_.write('yval_EVEN1_%s = ' % (MASS_STR))
            file2_.write(str(tprt3.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_test_EVEN1_%s = ' % (MASS_STR))
            file2_.write(str(y_pred_3.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_test_EVEN1_%s = ' % (MASS_STR))
            file2_.write(str(y_predS_3.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_train_ODD1_%s = ' % (MASS_STR))
            file2_.write(str(y_pred_train_3.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_train_ODD1_%s = ' % (MASS_STR))
            file2_.write(str(y_predS_train_3.tolist()))
            file2_.write('\n')
        else: ## Train->even; Test/val -> odd                                                                                                                                                                                                                           
            file2_.write('train_auc_EVEN1_%s = %0.8f\n' %  (MASS_STR, train_auc))
            file2_.write('test_auc_ODD1_%s  = %0.8f\n' % (MASS_STR, test_auct))
            file2_.write('xtrain_EVEN1_%s = ' % (MASS_STR))
            file2_.write(str(fpr3.tolist()))
            file2_.write('\n')
            file2_.write('ytrain_EVEN1_%s = ' % (MASS_STR))
            file2_.write(str(tpr3.tolist()))
            file2_.write('\n')
            file2_.write('xval_ODD1_%s = ' % (MASS_STR))
            file2_.write(str(fprt3.tolist()))
            file2_.write('\n')
            file2_.write('yval_ODD1_%s = ' % (MASS_STR))
            file2_.write(str(tprt3.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_test_ODD1_%s = ' % (MASS_STR))
            file2_.write(str(y_pred_3.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_test_ODD1_%s = ' % (MASS_STR))
            file2_.write(str(y_predS_3.tolist()))
            file2_.write('\n')
            file2_.write('y_pred_train_EVEN1_%s = ' % (MASS_STR))
            file2_.write(str(y_pred_train_3.tolist()))
            file2_.write('\n')
            file2_.write('y_predS_train_EVEN1_%s = ' % (MASS_STR))
            file2_.write(str(y_predS_train_3.tolist()))
            file2_.write('\n')
        file2_.write('##-------------------------------------------##\n')
        
        ## ---- ROC Curves ---- ##
        #ax3.plot(
        #    fpr3, tpr3,
        #    lw = 2, linestyle = styleline[dd + dd*1], color = colors_mass[2],
        #    label = order_train_name[dd] + ' Single mass train %s GeV (area = %0.3f)'%(str(mass), train_auc)
        #    )
        #s3 += 1
        #ax3.plot(
        #    fprt3, tprt3,
        #    lw = 2, linestyle = styleline[dd + 1 + + dd*1], color = colors_mass[2],
        #    label = order_train_name[dd] + ' Single mass test %s GeV (area = %0.3f)'%(str(mass), test_auct)
        #    )
        #s3 += 1
        
        
        ## Plot NN Output Histograms
        if dd == 0 : # Odd train, Even test
            dict_plot3 = [ ## Output Histogtams
                   [y_pred_3, "-", colorhot[mm],  str(mass)+" GeV test BKG (Even)"],
                   [y_predS_3, "-", colorcold[mm], str(mass)+" GeV test signal (Even)"],
                   [y_pred_train_3, "--", colorhot[mm], str(mass)+" GeV train BKG (Odd)" ],
                   [y_predS_train_3, "--", colorcold[mm],      str(mass)+" GeV train signal (Odd)"]
                ]
        else: # Even train, Odd test
            dict_plot3 = [ ## Output Histogtams
                   [y_pred_3, "-", colorhot[mm+1],  str(mass)+" GeV test BKG (Odd)"],
                   [y_predS_3, "-", colorcold[mm+1], str(mass)+" GeV test signal (Odd)"],
                   [y_pred_train_3, "--", colorhot[mm+1], str(mass)+" GeV train BKG (Even)" ],
                   [y_predS_train_3, "--", colorcold[mm+1], str(mass)+" GeV train signal (Even)"]
                ]
            
        for item in dict_plot3 :
            values1, bins, _ = ax1c.hist(
                item[0],
                ls=item[1], color = item[2],
                label=item[3],
                range=(0,1),
                **hist_params
                )
            normed = sum(y_pred_3)
            mid = 0.5*(bins[1:] + bins[:-1])
            err=np.sqrt(values1*normed)/normed # denominator is because plot is normalized
            plt.errorbar(mid, values1, yerr=err, fmt='none', color= item[2], ecolor= item[2], edgecolor=item[2], lw=2)
        del dict_plot3[:] ## Clearing the list
        
    file2_.close()
    
    #plt.yscale('log')
    ax1a.legend(loc='upper center', title="by mass ", fontsize = 'small')
    ax1b.legend(loc='upper center', title="by mass ", fontsize = 'small')
    ax1c.legend(loc='upper center', title="by mass ", fontsize = 'small')
    if dd == 0: # Odd train, Even test
        filename3a="{}/NN_Output_Odd_TrainOnAllExcept_{}_Even_TestOn_{}_w_gen_mHH_{}.pdf".format(channel, str(mass), str(mass), hyppar) ## For gen_mHH as input Var.
        fig1a.savefig(filename3a);
        filename3b="{}/NN_Output_Odd_TrainOnAllExcept_{}_Even_TestOn_{}_wo_gen_mHH_{}.pdf".format(channel, str(mass), str(mass), hyppar) ## For gen_mHH as input Var.
        fig1b.savefig(filename3b);
        filename3c="{}/NN_Output_Odd_TrainOnAllExcept_{}_Even_TestOn_{}_SingleMass_{}.pdf".format(channel, str(mass), str(mass), hyppar) ## For gen_mHH as input Var.
        fig1c.savefig(filename3c);
    else: # Even train, Odd test 
        filename3a="{}/NN_Output_Even_TrainOnAllExcept_{}_Odd_TestOn_{}_w_gen_mHH_{}.pdf".format(channel, str(mass), str(mass), hyppar) ## For gen_mHH as input Var.
        fig1a.savefig(filename3a);
        filename3b="{}/NN_Output_Even_TrainOnAllExcept_{}_Odd_TestOn_{}_wo_gen_mHH_{}.pdf".format(channel, str(mass), str(mass), hyppar) ## For gen_mHH as input Var.
        fig1b.savefig(filename3b);
        filename3c="{}/NN_Output_Even_TrainOnAllExcept_{}_Odd_TestOn_{}_SingleMass_{}.pdf".format(channel, str(mass), str(mass), hyppar) ## For gen_mHH as input Var.
        fig1c.savefig(filename3c);
       
'''       
ax1.set_ylim([0.0,1.0])
ax1.set_xlim([0.0,1.0])
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.legend(loc="lower right", fontsize = 'small')
ax1.grid()
filename4a="{}/ROC_Interpol_BDT_w_genmHH_{}.pdf".format(channel, hyppar) ## For gen_mHH as input Var.
#filename4="{}/ROC_by_mass_{}.pdf".format(channel, hyppar2) ## w/o gen_mHH as input Var.
plt2.savefig(filename4a);

ax2.set_ylim([0.0,1.0])
ax2.set_xlim([0.0,1.0])
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate')
ax2.legend(loc="upper left", fontsize = 'small')
ax2.grid()
filename4b="{}/ROC_Interpol_BDT_wo_genmHH_{}.pdf".format(channel, hyppar) ## For gen_mHH as input Var.
plt2.savefig(filename4b);

ax3.set_ylim([0.0,1.0])
ax3.set_xlim([0.0,1.0])
ax3.set_xlabel('False Positive Rate')
ax3.set_ylabel('True Positive Rate')
ax3.legend(loc="upper left", fontsize = 'small')
ax3.grid()
filename4c="{}/ROC_Single_Mass_wo_genmHH_{}.pdf".format(channel, hyppar) ## For gen_mHH as input Var.
plt2.savefig(filename4c);
### ---------------------------------------- #### 
'''        
            

In [ ]:
"""
output a training and export to .pb (to be used on cpp)
"""
print ("Date: ", time.asctime( time.localtime(time.time()) ))
nameout = "model_erase"

out = k_model_binary.model.save("test_"+nameout+"_"+hyppar+"_Odd.hdf5")
file = open(nameout+"_variables.log","w")
file.write(str(features)+"\n")
file.close()

out = k_model_binary2.model.save("test_"+nameout+"_"+hyppar+"_Even.hdf5")
file = open(nameout+"_variables.log","w")
file.write(str(features)+"\n")
file.close()


## This bellow does not work, easier to be done on the command line,
## in any case the bellow is a template how to run it
##!python ../test/convert_hdf5_2_pb.py --input "test_"+nameout+".hdf5" --output "test_"+nameout+".pb"

In [ ]:
"""
If you want to load a model to reconpute anything or check loading just substitute k_model --> k_model_loaded
It only loads hdf5 format
"""
from keras.models import load_model
k_model_loaded = load_model("test_model_2lss_ttH_3cat_no4mom_noSemi_v6.hdf5")

In [ ]:
## the next do correlation matrices with variables
import seaborn 

In [ ]:
for target in [0,1] :
    corr_mat = data.loc[(data['target']==0), features].astype(float).corr() #
    fig, ax = plt.subplots(figsize=(20, 12)) 
    seaborn.heatmap(corr_mat, square=True, ax=ax, vmin=-1., vmax=1.)
    if(target == 0) : filename4="{}/Signal_Correl_{}.pdf".format(channel, hyppar)
    else: filename4="{}/Background_Correl_{}.pdf".format(channel, hyppar)   
    plt.savefig(filename4);